Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [472]:
from idlelib.autocomplete import TRY_A
from random import choice

from IPython.core.display_functions import clear_output
from tqdm.auto import tqdm
from enum import Enum
from typing import NamedTuple
import numpy as np
import numpy.typing as npt

In [473]:
PUZZLE_DIM = 7
VALID_STATE = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

class Action(Enum):
    LEFT = [[0], [1]]
    RIGHT = [[0], [-1]]
    UP = [[1], [0]]
    DOWN = [[-1], [0]]

class Position(NamedTuple):
    x: int
    y: int

class Rect(NamedTuple):
    top_left: tuple[int, int]
    bottom_right: tuple[int, int]

In [474]:
def is_action_valid(state: npt.NDArray[np.int_], action: Action, action_bounds: Rect = Rect((-1, -1), (PUZZLE_DIM, PUZZLE_DIM))) -> bool:
    hole_pos = np.nonzero(state == 0)
    target_pos = tuple(np.add(hole_pos, action.value))

    return not (target_pos[0] == action_bounds[0][1] or target_pos[1] == action_bounds[0][0] or target_pos[0] == action_bounds[1][1] or target_pos[1] == action_bounds[1][0])

def available_actions(state: npt.NDArray[np.int_]) -> list[Action]:
    actions: list[Action] = []

    if is_action_valid(state, Action.LEFT):
        actions.append(Action.LEFT)
    if is_action_valid(state, Action.RIGHT):
        actions.append(Action.RIGHT)
    if is_action_valid(state, Action.UP):
        actions.append(Action.UP)
    if is_action_valid(state, Action.DOWN):
        actions.append(Action.DOWN)

    return actions

In [475]:
def get_valid_positions(state: npt.NDArray[np.int_]) -> npt.NDArray[np.bool_]:
    return np.array(state == VALID_STATE)

def get_valid_rows(state: npt.NDArray[np.int_]) -> int:
    if np.argwhere(np.all(get_valid_positions(state), axis=1) == 0).shape == (0, 1):
        return PUZZLE_DIM

    return int(np.cumsum(np.argwhere(np.all(get_valid_positions(state), axis=1) == 0))[0])

def get_valid_columns(state: npt.NDArray[np.int_]) -> int:
    if np.argwhere(np.all(get_valid_positions(state), axis=0) == 0).shape == (0, 1):
        return PUZZLE_DIM

    return int(np.cumsum(np.argwhere(np.all(get_valid_positions(state), axis=0) == 0))[0])

def is_valid(state: npt.NDArray[np.int_]) -> bool:
    return get_valid_rows(state) == PUZZLE_DIM and get_valid_columns(state) == PUZZLE_DIM

In [476]:
def do_action(state: npt.NDArray[np.int_], action: Action, action_bounds: Rect = Rect((-1, -1), (PUZZLE_DIM, PUZZLE_DIM))) -> np.ndarray:
    if not is_action_valid(state, action, action_bounds):
        return state

    new_state = state.copy()

    hole_pos = np.nonzero(new_state == 0)
    target_pos = tuple(np.add(hole_pos, action.value))

    new_state[hole_pos], new_state[target_pos] = new_state[target_pos], new_state[hole_pos]
    return new_state

In [477]:
def is_target_at_correct_position(state: npt.NDArray[np.int_], target: int) -> bool:
    return np.all(np.argwhere(state == target) == np.argwhere(VALID_STATE == target))

def place_target_in_position(state: npt.NDArray[np.int_], target: int, position: tuple[int, int], movement_bounds: Rect = Rect((-1, -1), (PUZZLE_DIM, PUZZLE_DIM)), verbose: bool = False) -> np.ndarray:
    if is_target_at_correct_position(state, target):
        return state

    new_state = state.copy()

    target_pos = np.array(position[::-1])
    if verbose:
        print('target_pos', target_pos)
        print('aligning gap')
    # Place gap at the edge of the region defined by the target current position and its correct position.
    while True:
        gap_position = tuple(np.argwhere(new_state == 0).flatten())[::-1]
        current_position = tuple(np.argwhere(new_state == target).flatten())[::-1]
        if (
            gap_position[0] == current_position[0] or gap_position[1] == current_position[1] or
            gap_position[0] == position[0] or gap_position[1] == position[1] or
            is_target_at_correct_position(new_state, target)
        ):
            break

        hole_pos = np.argwhere(new_state == 0).flatten()
        starting_pos = np.argwhere(new_state == target).flatten()

        if verbose:
            print(new_state)
            print('hole_pos', hole_pos)
            print('starting_pos', starting_pos)

        positions = np.array([hole_pos, starting_pos]).reshape(2, 2)

        min_values = np.min(positions, axis=0)
        max_values = np.max(positions, axis=0)

        min_y, min_x = tuple(min_values)
        max_y, max_x = tuple(max_values)

        min_x -= 1
        min_y -= 1
        max_x += 1
        max_y += 1

        if verbose:
            print('top_left', (min_x, min_y))
            print('bottom_right', (max_x, max_y))

        if min_x < movement_bounds.top_left[0]:
            min_x = movement_bounds.top_left[0]
        if min_y < movement_bounds.top_left[1]:
            min_y = movement_bounds.top_left[1]
        if max_x > movement_bounds.bottom_right[0]:
            max_x = movement_bounds.bottom_right[0]
        if max_y > movement_bounds.bottom_right[1]:
            max_y = movement_bounds.bottom_right[1]

        if max_x - min_x == 2:
            if max_x == PUZZLE_DIM:
                min_x -= 1
            else:
                max_x += 1
        if max_y - min_y == 2:
            if max_y == PUZZLE_DIM:
                min_y -= 1
            else:
                max_y += 1

        bounds = Rect((min_x, min_y), (max_x, max_y))
        if verbose:
            print('movement_bounds', movement_bounds)
            print('bounds', bounds)

        if not is_action_valid(new_state, Action.DOWN, bounds):
            new_state = do_action(new_state, Action.RIGHT, bounds)
            if verbose:
                print(new_state)

            gap_position = tuple(np.argwhere(new_state == 0).flatten())[::-1]
            current_position = tuple(np.argwhere(new_state == target).flatten())[::-1]
            if (
                gap_position[0] == current_position[0] or gap_position[1] == current_position[1] or
                gap_position[0] == position[0] or gap_position[1] == position[1] or
                is_target_at_correct_position(new_state, target)
            ):
                break

        if not is_action_valid(new_state, Action.LEFT, bounds):
            new_state = do_action(new_state, Action.DOWN, bounds)
            if verbose:
                print(new_state)

            gap_position = tuple(np.argwhere(new_state == 0).flatten())[::-1]
            current_position = tuple(np.argwhere(new_state == target).flatten())[::-1]
            if (
                gap_position[0] == current_position[0] or gap_position[1] == current_position[1] or
                gap_position[0] == position[0] or gap_position[1] == position[1] or
                is_target_at_correct_position(new_state, target)
            ):
                break

        if not is_action_valid(new_state, Action.UP, bounds):
            new_state = do_action(new_state, Action.LEFT, bounds)
            if verbose:
                print(new_state)

            gap_position = tuple(np.argwhere(new_state == 0).flatten())[::-1]
            current_position = tuple(np.argwhere(new_state == target).flatten())[::-1]
            if (
                gap_position[0] == current_position[0] or gap_position[1] == current_position[1] or
                gap_position[0] == position[0] or gap_position[1] == position[1] or
                is_target_at_correct_position(new_state, target)
            ):
                break

        if not is_action_valid(new_state, Action.RIGHT, bounds):
            new_state = do_action(new_state, Action.UP, bounds)
            if verbose:
                print(new_state)

            gap_position = tuple(np.argwhere(new_state == 0).flatten())[::-1]
            current_position = tuple(np.argwhere(new_state == target).flatten())[::-1]
            if (
                gap_position[0] == current_position[0] or gap_position[1] == current_position[1] or
                gap_position[0] == position[0] or gap_position[1] == position[1] or
                is_target_at_correct_position(new_state, target)
            ):
                break
    if verbose:
        print('gap aligned')
        print('aligning_target')

    # Place target in position
    while True:
        hole_pos = np.argwhere(new_state == 0).flatten()
        starting_pos = np.argwhere(new_state == target).flatten()

        if verbose:
            print(new_state)
            print('hole_pos', hole_pos)
            print('starting_pos', starting_pos)
            print('target_pos', target_pos)

        positions = np.array([hole_pos, starting_pos, target_pos]).reshape(3, 2)

        min_values = np.min(positions, axis=0)
        max_values = np.max(positions, axis=0)

        min_y, min_x = tuple(min_values)
        max_y, max_x = tuple(max_values)

        min_x -= 1
        min_y -= 1
        max_x += 1
        max_y += 1

        if verbose:
            print('top_left', (min_x, min_y))
            print('bottom_right', (max_x, max_y))

        if min_x < movement_bounds.top_left[0]:
            min_x = movement_bounds.top_left[0]
        if min_y < movement_bounds.top_left[1]:
            min_y = movement_bounds.top_left[1]
        if max_x > movement_bounds.bottom_right[0]:
            max_x = movement_bounds.bottom_right[0]
        if max_y > movement_bounds.bottom_right[1]:
            max_y = movement_bounds.bottom_right[1]

        if max_x - min_x == 2:
            if max_x == PUZZLE_DIM:
                min_x -= 1
            else:
                max_x += 1
        if max_y - min_y == 2:
            if max_y == PUZZLE_DIM:
                min_y -= 1
            else:
                max_y += 1

        bounds = Rect((min_x, min_y), (max_x, max_y))
        if verbose:
            print('movement_bounds', movement_bounds)
            print('bounds', bounds)

        if not is_action_valid(new_state, Action.DOWN, bounds):
            new_state = do_action(new_state, Action.RIGHT, bounds)
            if verbose:
                print(new_state)

            current_position = tuple(np.argwhere(new_state == target).flatten())[::-1]
            if current_position == position or is_target_at_correct_position(new_state, target):
                break

        if not is_action_valid(new_state, Action.LEFT, bounds):
            new_state = do_action(new_state, Action.DOWN, bounds)
            if verbose:
                print(new_state)

            current_position = tuple(np.argwhere(new_state == target).flatten())[::-1]
            if current_position == position or is_target_at_correct_position(new_state, target):
                break

        if not is_action_valid(new_state, Action.UP, bounds):
            new_state = do_action(new_state, Action.LEFT, bounds)
            if verbose:
                print(new_state)

            current_position = tuple(np.argwhere(new_state == target).flatten())[::-1]
            if current_position == position or is_target_at_correct_position(new_state, target):
                break

        if not is_action_valid(new_state, Action.RIGHT, bounds):
            new_state = do_action(new_state, Action.UP, bounds)
            if verbose:
                print(new_state)

            current_position = tuple(np.argwhere(new_state == target).flatten())[::-1]
            if current_position == position or is_target_at_correct_position(new_state, target):
                break
    if verbose:
        print('target_aligned')

    return new_state

def place_target_in_correct_position(state: npt.NDArray[np.int_], target: int, verbose=False) -> np.ndarray:
    target_y, target_x = tuple(np.argwhere(VALID_STATE == target).flatten())
    if is_target_at_correct_position(state, target):
        return state

    new_state = state.copy()

    building_row: bool = (get_valid_rows(new_state) == target_y and target_x != 0) or (target_x, target_y) == (0, 0)

    solving_region = Rect((target_x - 1, target_y - 1), (PUZZLE_DIM, PUZZLE_DIM))
    pivoting_region = Rect((get_valid_columns(new_state) - 1 + int(not building_row), get_valid_rows(new_state) - 1 + int(building_row and target_x != 0)), (PUZZLE_DIM, PUZZLE_DIM))

    if verbose:
        print('target', target)
        print('solving_region', solving_region)
        print('pivoting_region', pivoting_region)
        print('building_row', building_row)

    if target_x == PUZZLE_DIM - 1 or target_y == PUZZLE_DIM - 1:
        # Corner case

        # Position gap inside solving region
        new_state = do_action(new_state, Action.UP)
        if is_target_at_correct_position(new_state, target):
            return new_state

        new_state = do_action(new_state, Action.LEFT)
        if is_target_at_correct_position(new_state, target):
            return new_state

        offset = (-1, 1) if target_x == PUZZLE_DIM - 1 else (1, -1)
        pivot = tuple(np.add((target_x, target_y), offset))

        if verbose:
            print('pivot', pivot)

        # Placing target at pivot
        new_state = place_target_in_position(new_state, target, pivot, pivoting_region, verbose)

        # Placing gap at row below/column to the right of the pivot (building row/column)
        while True:
            gap_y, gap_x = tuple(np.argwhere(new_state == 0).flatten())

            if verbose:
                print(new_state)

            if building_row:
                if gap_y == pivot[1] + 1:
                    break

                if verbose:
                    print('aligning gap to target')

                if gap_y < pivot[1] + 1:
                    new_state = do_action(new_state, Action.UP)
                else:
                    new_state = do_action(new_state, Action.DOWN)
            else:
                if gap_x == pivot[0] + 1:
                    break

                if verbose:
                    print('aligning gap to target')

                if gap_x < pivot[0] + 1:
                    new_state = do_action(new_state, Action.LEFT)
                else:
                    new_state = do_action(new_state, Action.RIGHT)

        # Placing gap at cell to the left/above the pivot (building row/column)
        while True:
            gap_y, gap_x = tuple(np.argwhere(new_state == 0).flatten())

            if verbose:
                print('new_state')
                print(new_state)

            if building_row and gap_x == pivot[0] - 1 or not building_row and gap_y == pivot[1] - 1:
                break

            if verbose:
                print('adjusting gap')

            if building_row:
                new_state = do_action(new_state, Action.RIGHT)
            else:
                new_state = do_action(new_state, Action.DOWN)

        # Gap final placement
        if building_row:
            new_state = do_action(new_state, Action.DOWN)
        else:
            new_state = do_action(new_state, Action.RIGHT)

        # Fixed movements
        if building_row:
            new_state = do_action(new_state, Action.DOWN)
            new_state = do_action(new_state, Action.LEFT)
            new_state = do_action(new_state, Action.UP)
            new_state = do_action(new_state, Action.LEFT)
            new_state = do_action(new_state, Action.DOWN)
            new_state = do_action(new_state, Action.RIGHT)
            new_state = do_action(new_state, Action.RIGHT)
            new_state = do_action(new_state, Action.UP)
        else:
            new_state = do_action(new_state, Action.RIGHT)
            new_state = do_action(new_state, Action.UP)
            new_state = do_action(new_state, Action.LEFT)
            new_state = do_action(new_state, Action.UP)
            new_state = do_action(new_state, Action.RIGHT)
            new_state = do_action(new_state, Action.DOWN)
            new_state = do_action(new_state, Action.DOWN)
            new_state = do_action(new_state, Action.LEFT)

        if pivot == (PUZZLE_DIM - 2, PUZZLE_DIM - 2) and not building_row:
            new_state = do_action(new_state, Action.LEFT)
            new_state = do_action(new_state, Action.UP)

        return new_state

    current_y, current_x = tuple(np.argwhere(new_state == target).flatten())
    if current_x <= solving_region.top_left[0] or current_y <= solving_region.top_left[1]:
        # Position gap inside pivoting region
        while True:
            gap_y, gap_x = tuple(np.argwhere(new_state == 0).flatten())

            if verbose:
                print(new_state)

            if gap_x > pivoting_region.top_left[0] and gap_y > pivoting_region.top_left[1]:
                if verbose:
                    print('gap inside region')

                break

            if verbose:
                print('adjusting gap')
            new_state = do_action(new_state, Action.UP)
            if is_target_at_correct_position(new_state, target):
                return new_state

            new_state = do_action(new_state, Action.LEFT)
            if is_target_at_correct_position(new_state, target):
                return new_state

        if verbose:
            print('adjusting target')

        offset = (int(not building_row), int(building_row))
        target_pos = tuple(np.add((target_x, target_y), offset))

        if verbose:
            print('target_pos', target_pos)

        new_state = place_target_in_position(new_state, target, target_pos, pivoting_region, verbose)
        if is_target_at_correct_position(new_state, target):
            return new_state

    if verbose:
        print('target inside region')

    # Position gap inside solving region
    while True:
        gap_y, gap_x = tuple(np.argwhere(new_state == 0).flatten())

        if verbose:
            print('new_state')
            print(new_state)

        if gap_x > solving_region.top_left[0] and gap_y > solving_region.top_left[1]:
            if verbose:
                print('gap inside region')

            break

        if verbose:
            print('adjusting gap')

        new_state = do_action(new_state, Action.UP)
        if is_target_at_correct_position(new_state, target):
            return new_state

        new_state = do_action(new_state, Action.LEFT)
        if is_target_at_correct_position(new_state, target):
            return new_state

    current_y, current_x = tuple(np.argwhere(new_state == target).flatten())
    if current_x <= solving_region.top_left[0] or current_y <= solving_region.top_left[1]:
        # Position gap inside pivoting region
        while True:
            gap_y, gap_x = tuple(np.argwhere(new_state == 0).flatten())

            if verbose:
                print(new_state)

            if gap_x > pivoting_region.top_left[0] and gap_y > pivoting_region.top_left[1]:
                if verbose:
                    print('gap inside region')

                break

            if verbose:
                print('adjusting gap')
            new_state = do_action(new_state, Action.UP)
            if is_target_at_correct_position(new_state, target):
                return new_state

            new_state = do_action(new_state, Action.LEFT)
            if is_target_at_correct_position(new_state, target):
                return new_state

        if verbose:
            print('adjusting target')

        offset = (int(not building_row), int(building_row))
        target_pos = tuple(np.add((target_x, target_y), offset))

        if verbose:
            print('target_pos', target_pos)

        new_state = place_target_in_position(new_state, target, target_pos, pivoting_region, verbose)
        if is_target_at_correct_position(new_state, target):
            return new_state

    if verbose:
        print('target inside region')

    # Position gap inside solving region
    while True:
        gap_y, gap_x = tuple(np.argwhere(new_state == 0).flatten())

        if verbose:
            print('new_state')
            print(new_state)

        if gap_x > solving_region.top_left[0] and gap_y > solving_region.top_left[1]:
            if verbose:
                print('gap inside region')

            break

        if verbose:
            print('adjusting gap')

        new_state = do_action(new_state, Action.UP)
        if is_target_at_correct_position(new_state, target):
            return new_state

        new_state = do_action(new_state, Action.LEFT)
        if is_target_at_correct_position(new_state, target):
            return new_state

    new_state = place_target_in_position(new_state, target, (target_x, target_y), solving_region, verbose)

    return new_state

In [478]:
def solve(state: npt.NDArray[np.int_]):
    new_state = state.copy()

    for i in range(PUZZLE_DIM):
        new_state = place_target_in_correct_position(new_state, i + 1)

    for k in range(1, PUZZLE_DIM - 1):
        for i in range(PUZZLE_DIM - k):
            new_state = place_target_in_correct_position(new_state, k + PUZZLE_DIM * (i + k))
        for i in range(PUZZLE_DIM - k):
            new_state = place_target_in_correct_position(new_state, k + i + 1 + PUZZLE_DIM * k)

    new_state = place_target_in_correct_position(new_state, PUZZLE_DIM * PUZZLE_DIM - 1)

    return new_state

In [479]:
RANDOMIZE_STEPS = 100000

initial_state = VALID_STATE
for _ in range(RANDOMIZE_STEPS):
    initial_state = do_action(initial_state, choice(available_actions(initial_state)))

initial_state, solve(initial_state)

(array([[22, 27, 36, 12, 40, 34, 41],
        [ 4, 17, 25,  3, 44, 15,  8],
        [38, 48,  6, 33, 37, 42, 43],
        [13, 20,  2, 10, 46, 14, 31],
        [45, 21,  5, 28, 32,  9, 35],
        [26,  1, 19, 24, 30,  0, 16],
        [29, 11, 47, 39,  7, 23, 18]]),
 array([[ 1,  2,  3,  4,  5,  6,  7],
        [ 8,  9, 10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19, 20, 21],
        [22, 23, 24, 25, 26, 27, 28],
        [29, 30, 31, 32, 33, 34, 35],
        [36, 37, 38, 39, 40, 41, 42],
        [43, 44, 45, 46, 47, 48,  0]]))